# Hunting for restaurant locations in New York 

## Introduction/Business Problem

In this project, the task that we will deal with is finding an area in New York to open a restaurant. New York is a huge city, famous for offering any imaginable type of food. Location choice is crucial for such an endeavour, both in terms of the overall area and what type of possible customers it would attract, as well as the existing competition in the neighbourhood. 

## Data

From previous analysis we have a dataset of the neighbourhoods, their locations and the most common categories of venues for each of them. We have also performed some initial clustering based on the categories of venues available in each of them. This data can be used to provide possible neighbourhoods for further analysis, based on criteria that we will set according to the type of restaurant that we would like to open. 

We additionally can retrieve data of the venues ratings in order to analyse potential competition and find opportunities for our restaurant. 

## Methodology

A good location compromises of many factors. We are looking for areas that have a lot to offer in terms of various venues. We will thus create a weight score for each neighbourhood based on the types of venues located in each of them. As an example, neighbourhoods with parks, tourist attractions, shopping malls or bar are bound to be good choices for a restaurant. Filtering for the best scoring neighbourhoods, we will look for the ones where restaurants are not common and/or not as good rated. This should provide some valid options for locating the restaurant. 

## Execution


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

In [3]:
!conda install -c conda-forge geopy --yes # uncomment this line if needed

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if needed
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-0.25.3              |   py36hb3f55d8_0        11.4 MB  conda-forge
    tbb4py-2019.9              |   py36hc9558a2_0         245 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.7 MB

The following NEW packages will be INSTALLED:

    tbb:    2019.9-hc9558a2_0     conda-forge
    tbb4py: 2019.9-py36hc9558a2_0 conda-forge

The following packages will be UPDATED:

    pandas: 0.25.2-py36hb3f55d8_0 conda-forge --> 0

In [7]:
# get dataset
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [9]:
# store data of features 
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
# check the created dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [130]:
# will be needed later for the map visualisations
address = 'Manhattan, New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7896239, -73.9598939.


In [16]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [11]:
CLIENT_ID = 'IEELNVNQLNIMZOIGZZ0XTT2VFNIQ5SCCTJBZIHAJXJV5WMCM' # your Foursquare ID
CLIENT_SECRET = 'EI4ABXOSKG0DNPFBPPK133PMOQMWJYMUMB1Q5ZXO42WY4PUS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [173]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue ID', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [174]:
#run for all neibhbourhoods
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [175]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3313, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,4b4429abf964a52037f225e3,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,4baf59e8f964a520a6f93be3,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,4b79cc46f964a520c5122fe3,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,55f81cd2498ee903149fcc64,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,55f751ca498eacc0307d1cfe,Blink Fitness Riverdale,40.877147,-73.905837,Gym


### Analysis

In [22]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scen

In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scen

### Scoring 
We create a series of weights for each category of venues that we think would be in our favour to have near the location of our restaurant. We also add negative scores for competitive categories (other restaurants) 

In [203]:
# weighted scores per desired categories
category_weight={'Park':10, 'Coffee Shop': 5, 'Bar': 7, 'Hotel': 7, 'Theater': 10, 'Shopping Mall': 10, 'Exhibit': 9, 
                 'Nightclub': 10, 'Cocktail Bar': 7, 'Jazz Club': 7, 'Art Gallery': 5, 'Event Space': 5, 
                 'Basketball Court': 10, 'Baseball Field': 10, 'Tennis Stadium': 5, 
                 'Concert Hall': 10, 'Performing Arts Venue': 10, 'Korean Restaurant': -5, 'Japanese Restaurant': -5, 'Pizza Place': -5, 
                'French Restaurant': -5, 'Italian Restaurant': -5, 'New American Restaurant': -5, 'American Restaurant': -5}

In [204]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# top 10 venues categories per neighborhood 
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood','Score']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']
neighborhoods_venues_sorted['Score'] =0 

# loop rows 
for ind in np.arange(manhattan_grouped.shape[0]):
    # fill in top 10 categories per neighborhood 
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)
    # loop top 10 categories 
    for iCol in np.arange(num_top_venues):
        venue_category = neighborhoods_venues_sorted.iloc[ind, iCol+2]
        # if we have a score for this category
        if(venue_category in category_weight):
            rowScore = neighborhoods_venues_sorted.iloc[ind,1]
            rowScore = rowScore + category_weight[venue_category]
            neighborhoods_venues_sorted.iloc[ind,1] = rowScore
            #debugging - print(neighborhoods_venues_sorted.iloc[ind]['Neighborhood'], venue_category, category_weight[venue_category], rowScore)

# sort by Score
neighborhoods_venues_sorted = neighborhoods_venues_sorted.sort_values(by=['Score'], ascending=False)
# show what we have 
neighborhoods_venues_sorted.head(10)

,Neighborhood,Score,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Stuyvesant Town,49,Park,Bar,Boat or Ferry,Basketball Court,Baseball Field,Gym / Fitness Center,Cocktail Bar,Coffee Shop,Harbor / Marina,Bistro
17,Lincoln Square,30,Theater,Café,Plaza,Italian Restaurant,Gym / Fitness Center,Performing Arts Venue,Concert Hall,Park,French Restaurant,Indie Movie Theater
0,Battery Park City,27,Park,Coffee Shop,Hotel,Memorial Site,Gym,Wine Shop,Italian Restaurant,Playground,Food Court,Shopping Mall
3,Chelsea,22,Coffee Shop,Ice Cream Shop,Bakery,Italian Restaurant,Nightclub,Hotel,Theater,American Restaurant,Seafood Restaurant,Cycle Studio
22,Marble Hill,20,Coffee Shop,Sandwich Place,Seafood Restaurant,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Yoga Studio
23,Midtown,17,Hotel,Coffee Shop,Clothing Store,Sporting Goods Shop,Theater,Steakhouse,Japanese Restaurant,Spa,Bakery,Bookstore
35,Upper East Side,16,Exhibit,Italian Restaurant,Bakery,Art Gallery,Coffee Shop,Hotel,Gym / Fitness Center,French Restaurant,Juice Bar,Yoga Studio
37,Washington Heights,15,Café,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Bakery,Coffee Shop,Park,Mexican Restaurant,Supplement Shop,Tapas Restaurant
38,West Village,14,Italian Restaurant,New American Restaurant,Cosmetics Shop,Park,Wine Bar,American Restaurant,Cocktail Bar,Jazz Club,Bakery,Coffee Shop
9,Financial District,14,Coffee Shop,Pizza Place,Food Truck,American Restaurant,Hotel,Gym,Steakhouse,Wine Shop,Bar,Event Space


Some initial results come out with a top 10 of neighborhoods where French restaurants are not common.

### Clustering
We can split neighborhoods into clusters so we can see how our top 10 fits into related areas

In [205]:
# set number of clusters
kclusters = 6

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 4, 3, 0, 0, 1, 3, 0], dtype=int32)

In [206]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [207]:
print(manhattan_merged.shape)
manhattan_merged = manhattan_merged.sort_values(by=['Score'], ascending=False)
manhattan_10 = manhattan_merged.head(10) # isolate the top 10 

(40, 16)


Let's visualize the top 10 on a map, coloured by their cluster

In [208]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, width=800, height=400, tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, score in zip(manhattan_10['Latitude'], manhattan_10['Longitude'], manhattan_10['Neighborhood'], manhattan_10['Cluster Labels'], manhattan_10['Score']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster)+ ', Score: '+str(score), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],  
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [209]:
manhattan_10

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Score,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Manhattan,Stuyvesant Town,40.731000,-73.974052,0,49,Park,Bar,Boat or Ferry,Basketball Court,Baseball Field,Gym / Fitness Center,Cocktail Bar,Coffee Shop,Harbor / Marina,Bistro
13,Manhattan,Lincoln Square,40.773529,-73.985338,4,30,Theater,Café,Plaza,Italian Restaurant,Gym / Fitness Center,Performing Arts Venue,Concert Hall,Park,French Restaurant,Indie Movie Theater
28,Manhattan,Battery Park City,40.711932,-74.016869,3,27,Park,Coffee Shop,Hotel,Memorial Site,Gym,Wine Shop,Italian Restaurant,Playground,Food Court,Shopping Mall
17,Manhattan,Chelsea,40.744035,-74.003116,4,22,Coffee Shop,Ice Cream Shop,Bakery,Italian Restaurant,Nightclub,Hotel,Theater,American Restaurant,Seafood Restaurant,Cycle Studio
0,Manhattan,Marble Hill,40.876551,-73.910660,3,20,Coffee Shop,Sandwich Place,Seafood Restaurant,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Yoga Studio
15,Manhattan,Midtown,40.754691,-73.981669,0,17,Hotel,Coffee Shop,Clothing Store,Sporting Goods Shop,Theater,Steakhouse,Japanese Restaurant,Spa,Bakery,Bookstore
8,Manhattan,Upper East Side,40.775639,-73.960508,0,16,Exhibit,Italian Restaurant,Bakery,Art Gallery,Coffee Shop,Hotel,Gym / Fitness Center,French Restaurant,Juice Bar,Yoga Studio
2,Manhattan,Washington Heights,40.851903,-73.936900,1,15,Café,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Bakery,Coffee Shop,Park,Mexican Restaurant,Supplement Shop,Tapas Restaurant
29,Manhattan,Financial District,40.707107,-74.010665,0,14,Coffee Shop,Pizza Place,Food Truck,American Restaurant,Hotel,Gym,Steakhouse,Wine Shop,Bar,Event Space
24,Manhattan,West Village,40.734434,-74.006180,3,14,Italian Restaurant,New American Restaurant,Cosmetics Shop,Park,Wine Bar,American Restaurant,Cocktail Bar,Jazz Club,Bakery,Coffee Shop


In [210]:
#let's focus on the first cluster with highest scored area
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Score,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,49,Park,Bar,Boat or Ferry,Basketball Court,Baseball Field,Gym / Fitness Center,Cocktail Bar,Coffee Shop,Harbor / Marina,Bistro
15,Midtown,17,Hotel,Coffee Shop,Clothing Store,Sporting Goods Shop,Theater,Steakhouse,Japanese Restaurant,Spa,Bakery,Bookstore
8,Upper East Side,16,Exhibit,Italian Restaurant,Bakery,Art Gallery,Coffee Shop,Hotel,Gym / Fitness Center,French Restaurant,Juice Bar,Yoga Studio
29,Financial District,14,Coffee Shop,Pizza Place,Food Truck,American Restaurant,Hotel,Gym,Steakhouse,Wine Shop,Bar,Event Space
14,Clinton,12,Theater,Gym / Fitness Center,Italian Restaurant,Hotel,American Restaurant,Coffee Shop,Wine Shop,Spa,Sandwich Place,Gym
19,East Village,9,Bar,Chinese Restaurant,Wine Bar,Ice Cream Shop,Pizza Place,Mexican Restaurant,Cocktail Bar,Dessert Shop,Ramen Restaurant,Vegetarian / Vegan Restaurant
20,Lower East Side,7,Pizza Place,Café,Ramen Restaurant,Coffee Shop,Japanese Restaurant,Art Gallery,Cocktail Bar,Chinese Restaurant,Bakery,Farmers Market
33,Midtown South,4,Korean Restaurant,Hotel,Coffee Shop,Japanese Restaurant,Hotel Bar,Cosmetics Shop,American Restaurant,Boutique,Cocktail Bar,Gym / Fitness Center
32,Civic Center,4,Gym / Fitness Center,Coffee Shop,Hotel,French Restaurant,Italian Restaurant,Cocktail Bar,Sandwich Place,Bakery,Yoga Studio,American Restaurant
10,Lenox Hill,2,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Burger Joint,Cocktail Bar,Sporting Goods Shop,Café,Gym / Fitness Center,Gym


The clustering did not provide any further insights. Perhaps if we had combined additional data from other sources, this would be more valuable. 

### Finding competition via foursquare ratings
Let's check the top 3 neighborhoods for other restaurants

In [176]:
manhattan_venues.loc[manhattan_venues['Neighborhood'] == 'Stuyvesant Town'].loc[manhattan_venues['Venue Category'].str.contains('Restaurant')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category


In [211]:
manhattan_venues.loc[manhattan_venues['Neighborhood'] == 'Battery Park City'].loc[manhattan_venues['Venue Category'].str.contains('Restaurant')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
2245,Battery Park City,40.711932,-74.016869,535fc0ea498e4a159c49f7b1,Dig Inn,40.712835,-74.016013,American Restaurant
2261,Battery Park City,40.711932,-74.016869,556cb8c6498e751e51fbcf41,Parm,40.714514,-74.016264,Italian Restaurant
2263,Battery Park City,40.711932,-74.016869,53d3f607498edfb90a15baf5,Blue Ribbon Sushi Bar,40.712742,-74.016065,Sushi Restaurant
2271,Battery Park City,40.711932,-74.016869,52e2f72a498e1de9bce745b0,El Vez,40.714730,-74.015630,Mexican Restaurant
2297,Battery Park City,40.711932,-74.016869,4f0c7c106d866af8d40ae337,Atrio New York,40.715025,-74.015914,Mediterranean Restaurant
2299,Battery Park City,40.711932,-74.016869,57b4d7d7498e0b675b71f653,Osteria della Pace,40.710554,-74.012519,Italian Restaurant
2327,Battery Park City,40.711932,-74.016869,59792b2403369334be043a56,Vino e Grano,40.709953,-74.011574,Italian Restaurant
2329,Battery Park City,40.711932,-74.016869,5742850f498e0114292c6a52,Ningbo Cafe,40.707946,-74.018322,Chinese Restaurant


In [212]:
manhattan_venues.loc[manhattan_venues['Neighborhood'] == 'Lincoln Square'].loc[manhattan_venues['Venue Category'].str.contains('Restaurant')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
914,Lincoln Square,40.773529,-73.985338,4c881d60a0ffb60ccda81fc5,Lincoln Ristorante,40.773413,-73.983535,Italian Restaurant
916,Lincoln Square,40.773529,-73.985338,4b7c37c2f964a52026852fe3,The Grand Tier,40.773389,-73.983532,American Restaurant
917,Lincoln Square,40.773529,-73.985338,3fd66200f964a52034e81ee3,Rosa Mexicano,40.771183,-73.983325,Mexican Restaurant
920,Lincoln Square,40.773529,-73.985338,5078c4f1e4b0dad821e1f6bd,The Smith,40.771505,-73.982292,American Restaurant
923,Lincoln Square,40.773529,-73.985338,4da8daa5a86e771ea6fdad17,Boulud Sud,40.771930,-73.981576,Mediterranean Restaurant
936,Lincoln Square,40.773529,-73.985338,4c76c2e7947ca1cdb13f4537,Atlantic Grill,40.772077,-73.981455,Seafood Restaurant
938,Lincoln Square,40.773529,-73.985338,459395b7f964a52047401fe3,Cafe Fiorello,40.771735,-73.981996,Italian Restaurant
939,Lincoln Square,40.773529,-73.985338,478cd1f3f964a520444d1fe3,Bar Boulud,40.771859,-73.982301,French Restaurant
941,Lincoln Square,40.773529,-73.985338,4a2ff171f964a520f6981fe3,Nanoosh,40.775823,-73.982204,Mediterranean Restaurant
950,Lincoln Square,40.773529,-73.985338,414ccc00f964a520d21c1fe3,Cafe Luxembourg,40.777514,-73.983556,French Restaurant


Stuyvesant Town has no listed restaurants (since it is a residential area, this is to be expected), Battery Park City has several options by no French restaurant, and Lincoln Square is a highly crowded option. 

Let's consider the competition in _Battery Park City_

In [213]:
battery_restaurants=manhattan_venues.loc[manhattan_venues['Neighborhood'] == 'Battery Park City'].loc[manhattan_venues['Venue Category'].str.contains('Restaurant')]
battery_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
2245,Battery Park City,40.711932,-74.016869,535fc0ea498e4a159c49f7b1,Dig Inn,40.712835,-74.016013,American Restaurant
2261,Battery Park City,40.711932,-74.016869,556cb8c6498e751e51fbcf41,Parm,40.714514,-74.016264,Italian Restaurant
2263,Battery Park City,40.711932,-74.016869,53d3f607498edfb90a15baf5,Blue Ribbon Sushi Bar,40.712742,-74.016065,Sushi Restaurant
2271,Battery Park City,40.711932,-74.016869,52e2f72a498e1de9bce745b0,El Vez,40.714730,-74.015630,Mexican Restaurant
2297,Battery Park City,40.711932,-74.016869,4f0c7c106d866af8d40ae337,Atrio New York,40.715025,-74.015914,Mediterranean Restaurant
2299,Battery Park City,40.711932,-74.016869,57b4d7d7498e0b675b71f653,Osteria della Pace,40.710554,-74.012519,Italian Restaurant
2327,Battery Park City,40.711932,-74.016869,59792b2403369334be043a56,Vino e Grano,40.709953,-74.011574,Italian Restaurant
2329,Battery Park City,40.711932,-74.016869,5742850f498e0114292c6a52,Ningbo Cafe,40.707946,-74.018322,Chinese Restaurant


In [214]:
# initialise values
battery_restaurants['rating'] =0
battery_restaurants['price'] =0 
# loop venues
for ind in np.arange(battery_restaurants.shape[0]):
    br = battery_restaurants.iloc[ind]
    brID = br['Venue ID']
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            brID, CLIENT_ID, CLIENT_SECRET, VERSION)
    results = requests.get(url).json() 
    battery_restaurants.iloc[ind,8]=results['response']['venue']['rating']
    battery_restaurants.iloc[ind,9]=results['response']['venue']['price']['tier']
    print(results['response']['venue']['name'] , ' Rating: ', results['response']['venue']['rating'], ' Price: ', results['response']['venue']['price']['tier'] )
#

Dig Inn  Rating:  8.2  Price:  1
Parm  Rating:  8.2  Price:  2
Blue Ribbon Sushi Bar  Rating:  7.7  Price:  2
El Vez  Rating:  8.2  Price:  3
Atrio New York  Rating:  7.7  Price:  3
Osteria della Pace  Rating:  7.9  Price:  2
Vino e Grano  Rating:  7.5  Price:  2
Ningbo Cafe  Rating:  6.9  Price:  1


In [215]:
battery_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,rating,price
2245,Battery Park City,40.711932,-74.016869,535fc0ea498e4a159c49f7b1,Dig Inn,40.712835,-74.016013,American Restaurant,8.2,1
2261,Battery Park City,40.711932,-74.016869,556cb8c6498e751e51fbcf41,Parm,40.714514,-74.016264,Italian Restaurant,8.2,2
2263,Battery Park City,40.711932,-74.016869,53d3f607498edfb90a15baf5,Blue Ribbon Sushi Bar,40.712742,-74.016065,Sushi Restaurant,7.7,2
2271,Battery Park City,40.711932,-74.016869,52e2f72a498e1de9bce745b0,El Vez,40.714730,-74.015630,Mexican Restaurant,8.2,3
2297,Battery Park City,40.711932,-74.016869,4f0c7c106d866af8d40ae337,Atrio New York,40.715025,-74.015914,Mediterranean Restaurant,7.7,3
2299,Battery Park City,40.711932,-74.016869,57b4d7d7498e0b675b71f653,Osteria della Pace,40.710554,-74.012519,Italian Restaurant,7.9,2
2327,Battery Park City,40.711932,-74.016869,59792b2403369334be043a56,Vino e Grano,40.709953,-74.011574,Italian Restaurant,7.5,2
2329,Battery Park City,40.711932,-74.016869,5742850f498e0114292c6a52,Ningbo Cafe,40.707946,-74.018322,Chinese Restaurant,6.9,1


In [220]:
print ('Mean rating of restaurants in Battery Park: ', battery_restaurants['rating'].mean())
print ('Mean price tier of restaurants in Battery Park: ', battery_restaurants['price'].mean())

Mean rating of restaurants in Battery Park:  7.7875
Mean price tier of restaurants in Battery Park:  2.0


We can argue that there would be a possibility to have an expensive (price tier of 3) or even high-end (price of 4) restaurant, with a cuisine not existing in the area, such as French or Japanese. 

Discussion of the results is provided in the full report. 